# Binary Sentiment Analysis of French Movie Reviews

### Objectives
1. Text cleaning
2. Text preprocessing for custom embedding Neural Network
3. Train RNN model for sentiment analysis

⚠️ This notebook will be your final deliverable. 
- Make sure it can run "restart and run all"
- Delete useless code cells
- Do not "clear output"

# 0. Load data

Our dataset contains 30,000 french reviews of movies, along with the binary class 1 (positive) or 0 (negative) score

In [1]:
# We load the dataset for you
import pandas as pd
data = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies.csv')
data

review  polarity
0      Ça commence à devenir énervant d'avoir l'impre...         0
1      J'ai aimé ce film, si il ressemble a un docume...         1
2      Une grosse merde ce haneke ce faire produire p...         0
3      Beau mélodrame magnifiquement photographié, "V...         1
4      A la poursuite du diamant vers est un film pro...         1
...                                                  ...       ...
29946  Le meilleur film de super-héros derrière le ba...         1
29947  Un drame qui est d'une efficacité remarquable....         1
29948  Une daube hollywoodienne de plus, aucun intérê...         0
29949  Et voilà un nouveau biopic sur la star du X Li...         0
29950  Un film qui fait vieux, avec des acteurs pas t...         0

[29951 rows x 2 columns]

In [2]:
# We create features
y = data.polarity
X = data.review

# We analyse class balance
print(pd.value_counts(y))

1    15051
0    14900
Name: polarity, dtype: int64


In [3]:
# We check various reviews
print(f'polarity: {y[0]} \n')
print(X[0])

polarity: 0 

Ça commence à devenir énervant d'avoir l'impression de voir et revoir le même genre de film à savoir : la comédie romantique, surement le genre le plus prolifique de le production française actuelle. Le problème c'est que l'on a souvent affaire à des niaiseries de faible niveau comme celui ci. Avec un scenario ultra balisé et conventionnel, c'est à se demander comment ça peut passer les portes d'un producteur. Bref cette sempiternel histoire d'un homme mentant au nom de l'amour pour reconquérir une femme et qui à la fin se prend son mensonge en pleine figure est d'une originalité affligeante, et ce n'est pas la présence au casting de l'ex miss météo Charlotte Le Bon qui rêve surement d'avoir la même carrière que Louise Bourgoin qui change la donne.


# 1. Clean Text

❓ We need to give a _quick & dirty_ cleaning to all the sentences in the dataset. Create a variable `X_clean` of similar shape, but with the following cleaning:
- Replace french accents by their non-accentuated equivalent using the [unidecode.unidecode()](https://pypi.org/project/Unidecode/) method
- Reduce all uppercases to lowercases
- Remove any characters outside of a-z, for instance using `string.isalpha()`

😌 You will be given the solution `X_clean` in the next question to make sure you can complete the challenge

In [4]:
type(X)

pandas.core.series.Series

In [5]:
len(X)

29951

In [6]:
X

0        Ça commence à devenir énervant d'avoir l'impre...
1        J'ai aimé ce film, si il ressemble a un docume...
2        Une grosse merde ce haneke ce faire produire p...
3        Beau mélodrame magnifiquement photographié, "V...
4        A la poursuite du diamant vers est un film pro...
                               ...                        
29946    Le meilleur film de super-héros derrière le ba...
29947    Un drame qui est d'une efficacité remarquable....
29948    Une daube hollywoodienne de plus, aucun intérê...
29949    Et voilà un nouveau biopic sur la star du X Li...
29950    Un film qui fait vieux, avec des acteurs pas t...
Name: review, Length: 29951, dtype: object

In [7]:
X

0        Ça commence à devenir énervant d'avoir l'impre...
1        J'ai aimé ce film, si il ressemble a un docume...
2        Une grosse merde ce haneke ce faire produire p...
3        Beau mélodrame magnifiquement photographié, "V...
4        A la poursuite du diamant vers est un film pro...
                               ...                        
29946    Le meilleur film de super-héros derrière le ba...
29947    Un drame qui est d'une efficacité remarquable....
29948    Une daube hollywoodienne de plus, aucun intérê...
29949    Et voilà un nouveau biopic sur la star du X Li...
29950    Un film qui fait vieux, avec des acteurs pas t...
Name: review, Length: 29951, dtype: object

In [8]:
### YOUR CODE
from unidecode import unidecode
X_clean = X
for i in range(len(X)):
    X_clean[i] = unidecode(X[i]).lower()
    cleanlist = []
    for j in X_clean[i]:
        if j.isalpha() or j == " ":
            cleanlist.append(j)
    X_clean[i] = "".join(cleanlist)           

<ipython-input-8-d246336fc18d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_clean[i] = unidecode(X[i]).lower()
<ipython-input-8-d246336fc18d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_clean[i] = "".join(cleanlist)


In [9]:
X_clean

0        ca commence a devenir enervant davoir limpress...
1        jai aime ce film si il ressemble a un document...
2        une grosse merde ce haneke ce faire produire p...
3        beau melodrame magnifiquement photographie ver...
4        a la poursuite du diamant vers est un film pro...
                               ...                        
29946    le meilleur film de superheros derriere le bat...
29947    un drame qui est dune efficacite remarquable u...
29948    une daube hollywoodienne de plus aucun interet...
29949    et voila un nouveau biopic sur la star du x li...
29950    un film qui fait vieux avec des acteurs pas to...
Name: review, Length: 29951, dtype: object

In [10]:
from nbresult import ChallengeResult

result = ChallengeResult('C14',
    shape = X_clean.shape,
    first_sentence = X_clean[0]
)
result.write()

# 2. Preprocess data

Now that we have clean sentences, we need to convert each one into a list of integers of fixed size
- For example, the sentence: `"this was good"` should become something like `array([1, 3, 18, 0, 0, 0, ...0], dtype=int32)` where each integer match to a each _unique_ word in your corpus of sentences.

❓ Create a numpy ndarray `X_input` of shape (29951, 100) that will be the direct input to your Neutral Network. 

- 29951 represents the number of reviews in the dataset `X_clean`
- 100 represents the maximum number of words to keep for each movie review.
- It must contain only numerical values, without any `NaN`
- In the process, compute and save the number of _unique_ words in your cleaned corpus under `vocab_size` variable

👉 First, you **must** start back from the clean solution below (14Mo)

In [11]:
X_clean = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_clean.csv")['review']
X_clean

0        ca commence a devenir enervant de voir et revo...
1        aime ce film si il ressemble a un documentaire...
2        une grosse merde ce haneke ce faire produire p...
3        beau melodrame magnifiquement photographie ver...
4        a la poursuite du diamant vers est un film pro...
                               ...                        
29946    le meilleur film de derriere le batman de nola...
29947    un drame qui est efficacite remarquable un fil...
29948    une daube hollywoodienne de plus aucun interet...
29949    et voila un nouveau biopic sur la star du x li...
29950    un film qui fait vieux avec des acteurs pas to...
Name: review, Length: 29951, dtype: object

In [12]:
import numpy as np
from collections import defaultdict
from itertools import count
X_inputlist = []
word_to_index = defaultdict(count(1).__next__)
for i in range (29951):  
    sentence = X_clean[i]
    wordlist = []
    counter = 0
    for word in sentence.split():
        
        wordlist.append(word_to_index[word])
        counter +=1
        if counter == 100:
            break
    if counter < 100:
        for j in range (counter,100):
            wordlist.append(0)
    X_inputlist.append(wordlist)
X_input = np.array(X_inputlist)

In [13]:
X_input.shape

(29951, 100)

In [14]:
X_input

array([[    1,     2,     3, ...,     6,    76,    77],
       [   78,    72,    13, ...,     0,     0,     0],
       [   59,    94,    95, ...,     0,     0,     0],
       ...,
       [   59,  2043,  4915, ...,     0,     0,     0],
       [    8,  1052,    36, ..., 40281, 54160,    10],
       [   36,    13,    61, ...,  7760,    25,    10]])

In [21]:
print(X_input.max())

62379


In [22]:
vocab_size = X_input.max()

In [23]:
from nbresult import ChallengeResult

result = ChallengeResult('C1415',
    type_X = type(X_input),
    shape = X_input.shape, 
    input_1 = X_input[1], 
)
result.write()

# 3. Neural Network

❓Create and fit a Neural Netork that takes `X_input` and `y` as input, to binary classify each sentence's sentiment

- You cannot use transfer learning or other pre-existing Word2Vec models
- You must use a "recurrent" architecture to _capture_ a notion of order in the sentences' words
- The performance metrics for this task is "accuracy"
- Store your model in a variable `model` 
- Store the result your `model.fit()` in a variable `history`. 
- ⚠️ `history.history` must comprises a measure of the `val_accuracy` at each epoch.
- You don't need to cross-validate your model

😌 Don't worry, you will not be judged on your computer power: You should be able to reach accuracy significantly better than baseline in less than 3 minutes even without GPUs.

👉 But first, you **must** start back from the solution below (70Mo)

In [18]:
url = 'https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/movies_X_input.csv'
X_input = np.genfromtxt(url, delimiter=',', dtype='int32')

In [32]:
## YOUR CODE
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X_input, y, test_size=0.33)

model = Sequential()
model.add(layers.Embedding(input_dim=62380, output_dim=1))   
                                                       
model.add(layers.LSTM(68, activation='relu'))
model.add(layers.Dense(48, activation='relu'))
model.add(layers.Dense(48, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, 
          epochs=20, batch_size=16,validation_data=(X_test, y_test))

Epoch 1/20
1255/1255 [==============================] - 57s 45ms/step - loss: 0.6932 - accuracy: 0.4969 - val_loss: 0.6931 - val_accuracy: 0.4990
Epoch 2/20
1255/1255 [==============================] - 59s 47ms/step - loss: 0.6932 - accuracy: 0.4926 - val_loss: 0.6931 - val_accuracy: 0.5010
Epoch 3/20
1255/1255 [==============================] - 63s 50ms/step - loss: 0.6932 - accuracy: 0.4981 - val_loss: 0.6931 - val_accuracy: 0.4990
Epoch 4/20
1255/1255 [==============================] - 64s 51ms/step - loss: 0.6932 - accuracy: 0.5009 - val_loss: 0.6931 - val_accuracy: 0.5010
Epoch 5/20
1255/1255 [==============================] - 66s 52ms/step - loss: 0.6932 - accuracy: 0.4950 - val_loss: 0.6931 - val_accuracy: 0.4990
Epoch 6/20
1255/1255 [==============================] - 66s 53ms/step - loss: 0.6932 - accuracy: 0.4999 - val_loss: 0.6931 - val_accuracy: 0.5010
Epoch 7/20
1255/1255 [==============================] - 67s 54ms/step - loss: 0.6932 - accuracy: 0.4912 - val_loss: 0.6931 -

In [33]:
from nbresult import ChallengeResult
result = ChallengeResult('C1517',
                         history=history.history)
result.write()